<a href="https://colab.research.google.com/github/jagannath311/DataScience-/blob/master/crf_411746.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sklearn_crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split

In [0]:
def readDataset(filename):
  out=[]
  sent=[]
  lines=open(filename).readlines()
  for line in lines:
    line=line.strip()
    if line=='':
      out.append(sent)
      sent=[]
    else:
      words=line.split()
      sent.append((words[0],words[1]))
  if(len(sent)!=0):
    out.append(sent)
  return out

In [0]:
sent_gnb=readDataset('predicted_words_gnb.txt')
sent_mlp=readDataset('predicted_words_mlp.txt')
sent_entMax=readDataset('predicted_words_l2.txt')


In [0]:
def extract_features(sentence,position):
  features=[]
  features.append('CurrentWord:'+sentence[position][0])
  if(position<len(sentence)-1):
    features.append('NextWord:'+sentence[position+1][0])
  if(position<len(sentence)-2):
    features.append('NexttoNextWord:'+sentence[position+2][0])
  if(position>0):
    features.append('prevWord:'+sentence[position-1][0])
  if(position-1>0):
    features.append('prevtoprevword:'+sentence[position-2][0])
  if(position==0):
    features.append('BOS')
  if(position==len(sentence)-1):
    features.append('EOS')
  return features
def extract_sentence_features(sentence):
  return [extract_features(sentence,pos) for pos in range(len(sentence))]
def extract_dataset_features(dataset):
  return [extract_sentence_features(dataset[sent]) for sent in range(len(dataset))]


In [0]:
  def extract_sentence_labels(sentence):
    return [sentence[pos][1] for pos in range(len(sentence))]
  def extract_dataset_labels(dataset):
    return [extract_sentence_labels(dataset[sent]) for sent in range(len(dataset))]

In [0]:
x_gnb=extract_dataset_features(sent_gnb)
y_gnb=extract_dataset_labels(sent_gnb)
x_mlp=extract_dataset_features(sent_mlp)
y_mlp=extract_dataset_labels(sent_mlp)
x_entMax=extract_dataset_features(sent_entMax)
y_entMax=extract_dataset_labels(sent_entMax)

train_x_gnb,test_x_gnb,train_y_gnb,test_y_gnb=train_test_split(x_entMax,y_entMax,test_size=0.2,random_state=1234)
train_x_mlp,test_x_mlp,train_y_mlp,test_y_mlp=train_test_split(x_mlp,y_mlp,test_size=0.2,random_state=1234)
train_x_entMax,test_x_entMax,train_y_entMax,test_y_entMax=train_test_split(x_entMax,y_entMax,test_size=0.2,random_state=1234)

In [0]:
crf_gnb=sklearn_crfsuite.CRF(algorithm='lbfgs',all_possible_states=True,all_possible_transitions=True,c1=0.1,c2=0.1,max_iterations=100)
crf_gnb.fit(train_x_gnb,train_y_gnb)
crf_mlp=sklearn_crfsuite.CRF(algorithm='lbfgs',all_possible_states=True,all_possible_transitions=True,c1=0.1,c2=0.1,max_iterations=100)
crf_mlp.fit(train_x_mlp,train_y_mlp)
crf_entMax=sklearn_crfsuite.CRF(algorithm='lbfgs',all_possible_states=True,all_possible_transitions=True,c1=0.1,c2=0.1,max_iterations=100)
crf_entMax.fit(train_x_entMax,train_y_entMax)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=True, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
label_gnb=list(crf_gnb.classes_)
label_mlp=list(crf_mlp.classes_)
label_entMax=list(crf_entMax.classes_)

In [0]:
test_pred_gnb=crf_gnb.predict(test_x_gnb)
metrics.flat_f1_score(test_y_gnb,test_pred_gnb,average='weighted',labels=label_gnb)
metrics.flat_classification_report(test_y_gnb,test_pred_gnb,labels=label_gnb,digits=3)

In [0]:
test_pred_mlp=crf_mlp.predict(test_x_mlp)
metrics.flat_f1_score(test_y_mlp,test_pred_mlp,average='weighted',labels=label_mlp)
metrics.flat_classification_report(test_y_mlp,test_pred_mlp,labels=label_mlp,digits=3)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n          bn      0.000     0.000     0.000         5\n          en      0.660     0.493     0.564        67\n          ml      0.000     0.000     0.000         0\n          ta      0.000     0.000     0.000         0\n          hi      0.534     0.765     0.629        51\n          kn      0.000     0.000     0.000         0\n          te      0.000     0.000     0.000         0\n\n   micro avg      0.585     0.585     0.585       123\n   macro avg      0.171     0.180     0.170       123\nweighted avg      0.581     0.585     0.568       123\n'

In [0]:
test_pred_entMax=crf_entMax.predict(test_x_entMax)
metrics.flat_f1_score(test_y_entMax,test_pred_entMax,average='weighted',labels=label_entMax)
metrics.flat_classification_report(test_y_entMax,test_pred_entMax,labels=label_entMax,digits=3)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n          en      0.660     0.493     0.564        67\n          hi      0.534     0.765     0.629        51\n          ml      0.000     0.000     0.000         0\n          kn      0.000     0.000     0.000         0\n          bn      0.000     0.000     0.000         5\n          te      0.000     0.000     0.000         0\n\n   micro avg      0.585     0.585     0.585       123\n   macro avg      0.199     0.210     0.199       123\nweighted avg      0.581     0.585     0.568       123\n'